This script reads a plan(with multiple line details for multiple days) and a schedule from an excel file and adds/ overwrite a target_kpi file considering the hourly kpi distribution semantically.

In [1]:
!pip install pandas openpyxl

In [13]:
# all imports
import pandas as pd
import numpy as np
import random
from random import randint
from google.colab import drive

In [15]:
# Read plan and schedule data from drive
drive.mount('/content/drive')
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
plan.head()

,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,2096,660,23,5,5,4,28,8.28,17354.88
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1859,660,23,5,5,4,28,9.45,17567.55
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,1792,660,20,5,5,4,25,8.28,14837.76
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,1807,660,23,5,5,4,28,10.11,18268.77
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,2167,660,23,5,5,4,28,8.21,17791.07


In [17]:
schedule

,hour,start_time,end_time
0,1,07:30:00,08:30:00
1,2,08:30:00,09:30:00
2,3,09:30:00,10:30:00
3,4,10:30:00,11:30:00
4,5,11:30:00,13:00:00
5,6,13:00:00,14:00:00
6,7,14:00:00,15:00:00
7,8,15:00:00,16:00:00
8,9,16:00:00,17:00:00
9,10,17:00:00,18:00:00


In [18]:
# Generate the skeleton of the target table
repeated_plan_date =  plan['date'].repeat(len(schedule)).reset_index(drop=True)
repeated_module_id = plan['module_id'].repeat(len(schedule)).reset_index(drop=True)
hours = list(range(len(schedule)+1))
hours.remove(0)
repeated_hour = hours*len(plan)
targets = pd.DataFrame({'Date': repeated_plan_date, 'Line' : repeated_module_id,
                        'Hour': repeated_hour})




In [19]:
targets

,Date,Line,Hour
0,2024-07-26,F1M7,1
1,2024-07-26,F1M7,2
2,2024-07-26,F1M7,3
3,2024-07-26,F1M7,4
4,2024-07-26,F1M7,5
...,...,...,...
149,2024-08-01,F1M8,7
150,2024-08-01,F1M8,8
151,2024-08-01,F1M8,9
152,2024-08-01,F1M8,10


In [7]:
random.seed(42)

In [20]:

def distribute_total(total_qty, num_hours):
  """
    Distribute a given number of total qty between a specified number
    of hours.

    :param total_qty: The total qty to be distributed.
    :param num_hours: The number of hours.
    :return: A list with the distributed values.
  """
  # Initial distribution: assign each event a base value

  base_qty = total_qty // num_hours
  remaining = total_qty % num_hours

  distribution = [base_qty] * num_hours

  # Randomly distribute the remaining value
  for _ in range(remaining):
      distribution[np.random.randint(0, num_hours)] += 1

  return distribution

In [21]:
def generate_hourly_counts(targets, plan, line_quality_regulator):
    """
    Generate counts for each hour of each day for each line group of a DataFrame
    to match the target counts by distributing the target sum among all rows in
    the hour + date + line group.

    :param target: DataFrame containing targets.
    :param plan: DataFrame with planned values per day.
    :param line_quality_regulator: Variable determining what is the number of
    maximum pcs a line fails to produce from a given daily target ,
    given as a dictionary.
    :return: DataFrame with generated counts.
    """
    # Merge the dataframes on the 'hour, date, line' columns
    merged_df = targets.merge(plan[['date', 'module_id','qty', 'mins', 'operator', 'helper',
       'ironman', 'qc', 'manpower', 'smv', 'produced_mins']],
                              left_on=['Date', 'Line'],
                              right_on= ['date','module_id'], how='left')
    merged_df.drop(columns=['date', 'module_id'], inplace=True)


    # Create a new column for generated counts
    #merged_df['generated_qty'] = 0


    # Iterate over each date, hour group
    for (date, line), group in merged_df.groupby(['Date', 'Line']):
        # Get the target sum for this date + line
        target_sum = group['qty'].iloc[0]

        # Generate the new counts by distributing the target sum
        generated_counts = distribute_total(target_sum, len(group))

        # Adjust generated counts
        adjusted_counts = [count - randint(0, line_quality_regulator[line])
        for count in generated_counts]

        merged_df.loc[group.index, 'FTT'] = adjusted_counts

    # # # Drop the 'target_count' column as it's no longer needed
    # # result_df = merged_df.drop(columns=['target_count'])

    return merged_df

In [22]:
# Generate FTTs close(but lesser in sum) to the daily target qty
line_quality_regulator = {'F1M7': 4, 'F1M8': 10}
merged = generate_hourly_counts(targets, plan,line_quality_regulator)
merged['FTT'] = merged['FTT'].astype(int)
merged

,Date,Line,Hour,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins,FTT
0,2024-07-26,F1M7,1,2096,660,23,5,5,4,28,8.28,17354.88,186
1,2024-07-26,F1M7,2,2096,660,23,5,5,4,28,8.28,17354.88,187
2,2024-07-26,F1M7,3,2096,660,23,5,5,4,28,8.28,17354.88,192
3,2024-07-26,F1M7,4,2096,660,23,5,5,4,28,8.28,17354.88,190
4,2024-07-26,F1M7,5,2096,660,23,5,5,4,28,8.28,17354.88,188
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,1407,660,14,6,5,4,20,8.28,11649.96,120
150,2024-08-01,F1M8,8,1407,660,14,6,5,4,20,8.28,11649.96,126
151,2024-08-01,F1M8,9,1407,660,14,6,5,4,20,8.28,11649.96,127
152,2024-08-01,F1M8,10,1407,660,14,6,5,4,20,8.28,11649.96,128


In [23]:
#Convert start_time and end_time to datetime
schedule['start_time'] = pd.to_datetime(schedule['start_time'], format='%H:%M:%S')
schedule['end_time'] = pd.to_datetime(schedule['end_time'], format='%H:%M:%S')

# Calculate the duration between start_time and end_time in hours
schedule['clock_hours'] = (schedule['end_time'] - schedule['start_time']).dt.total_seconds() / 3600

# Sum the clock_hours to get total hours in clock time
total_clock_hours = schedule['clock_hours'].sum()
total_clock_hours

11.5

In [24]:
# Transforms for the target dataframe
merged.rename(columns={'smv': 'SMV'}, inplace=True)
merged['TPPM'] = round(merged['qty'] / ((total_clock_hours)*60),2)
merged


,Date,Line,Hour,qty,mins,operator,helper,ironman,qc,manpower,SMV,produced_mins,FTT,TPPM
0,2024-07-26,F1M7,1,2096,660,23,5,5,4,28,8.28,17354.88,186,3.04
1,2024-07-26,F1M7,2,2096,660,23,5,5,4,28,8.28,17354.88,187,3.04
2,2024-07-26,F1M7,3,2096,660,23,5,5,4,28,8.28,17354.88,192,3.04
3,2024-07-26,F1M7,4,2096,660,23,5,5,4,28,8.28,17354.88,190,3.04
4,2024-07-26,F1M7,5,2096,660,23,5,5,4,28,8.28,17354.88,188,3.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,1407,660,14,6,5,4,20,8.28,11649.96,120,2.04
150,2024-08-01,F1M8,8,1407,660,14,6,5,4,20,8.28,11649.96,126,2.04
151,2024-08-01,F1M8,9,1407,660,14,6,5,4,20,8.28,11649.96,127,2.04
152,2024-08-01,F1M8,10,1407,660,14,6,5,4,20,8.28,11649.96,128,2.04


In [25]:
def generate_defective_kpis(merged, defective_quality_regulator, defect_regulator,
                            rectifying_regulator, line_wastage_regulator):
  """
  Generate defective KPIs for a given DataFrame statistically
  based on the provided parameters.

  :param merged: skeleton df for targets
  :param defective_quality_regulator: Maximum number of defective possible
  for the line of concern
  :param defect_regulator: Maximum difference between defectives and defects
  possible for the line of concern
  :param rectifying_regulator: Maximum pcs that would not be possible to be
  rectified froma given defective count for the line of concern
  param line_wastage_regulator: Maximum pcs that are not considred defective
  that will be rejected starightaway from the line of concern
  :return: None

  """
  merged.loc[:, 'Defective'] = np.random.randint(0,defective_quality_regulator,
                                                 len(merged))
  merged.loc[merged['Defective'] == 0, 'Defect'] = 0
  merged.loc[merged['Defective'] == 0, 'Rectified'] = 0

  merged.loc[merged['Defective'] > 0, 'Defect'] = merged.loc[merged['Defective'] > 0,
                                                             'Defective'] + np.random.randint(0,defect_regulator,
                                                                                              len(merged[merged['Defective'] > 0]))
  merged.loc[merged['Defective'] > 0, 'Rectified'] = merged.loc[merged['Defective'] > 0,
                                                                'Defective'] - np.random.randint(0,rectifying_regulator, len(merged[merged['Defective'] > 0]))
  merged.loc[merged['Rectified'] < 0, 'Rectified'] = 0

  merged.loc[:, 'Rejet'] = merged['Defective'] - merged['Rectified'] + np.random.randint(0,line_wastage_regulator, len(merged))
  return merged

In [26]:
defective_quality_regulator = {'F1M7': 10, 'F1M8': 10}
defect_regulator = {'F1M7': 4, 'F1M8': 5}
rectifying_regulator = {'F1M7': 4, 'F1M8': 3}
line_wastage_regulator = {'F1M7': 3, 'F1M8': 2}
modules = merged['Line'].unique()
new_merged = pd.DataFrame()
for module in modules:
  modulewise_df = merged[merged['Line']== module]
  result = generate_defective_kpis(modulewise_df, defective_quality_regulator[module],
                          defect_regulator[module],
                          rectifying_regulator[module],
                          line_wastage_regulator[module]
                          )
  new_merged = pd.concat([new_merged, result])
  new_merged.reset_index(drop=True, inplace=True)
new_merged


<ipython-input-25-74ec73a7b294>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.loc[:, 'Defective'] = np.random.randint(0,defective_quality_regulator,
<ipython-input-25-74ec73a7b294>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged.loc[merged['Defective'] == 0, 'Defect'] = 0
<ipython-input-25-74ec73a7b294>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,Date,Line,Hour,qty,mins,operator,helper,ironman,qc,manpower,SMV,produced_mins,FTT,TPPM,Defective,Defect,Rectified,Rejet
0,2024-07-26,F1M7,1,2096,660,23,5,5,4,28,8.28,17354.88,186,3.04,2,4.0,0.0,3.0
1,2024-07-26,F1M7,2,2096,660,23,5,5,4,28,8.28,17354.88,187,3.04,9,12.0,7.0,2.0
2,2024-07-26,F1M7,3,2096,660,23,5,5,4,28,8.28,17354.88,192,3.04,5,6.0,4.0,3.0
3,2024-07-26,F1M7,4,2096,660,23,5,5,4,28,8.28,17354.88,190,3.04,2,5.0,0.0,2.0
4,2024-07-26,F1M7,5,2096,660,23,5,5,4,28,8.28,17354.88,188,3.04,4,6.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,1407,660,14,6,5,4,20,8.28,11649.96,120,2.04,2,4.0,0.0,3.0
150,2024-08-01,F1M8,8,1407,660,14,6,5,4,20,8.28,11649.96,126,2.04,5,6.0,4.0,1.0
151,2024-08-01,F1M8,9,1407,660,14,6,5,4,20,8.28,11649.96,127,2.04,5,9.0,4.0,2.0
152,2024-08-01,F1M8,10,1407,660,14,6,5,4,20,8.28,11649.96,128,2.04,5,5.0,3.0,2.0


In [27]:
new_merged['Defect'] = new_merged['Defect'].astype(int)
new_merged['Rectified'] = new_merged['Rectified'].astype(int)
new_merged['Reject'] = new_merged['Rejet'].astype(int)

In [29]:
def generate_manpower_kpis(merged, manpower_regulator):
  """
  Generate manpower KPIs for a given DataFrame statistically
  based on the provided parameters.

  :param merged: skeleton df for targets
  :param manpower_regulator: Maximum number of operator/helper variation possible
  :return: None

  """
  merged['Operator'] = (merged['operator'] - np.random.choice([0,manpower_regulator],
                                          len(merged), p=[0.95, 0.05])).astype(int)
  merged['Helper'] = (merged['helper'] - np.random.choice([0,manpower_regulator],
                                          len(merged), p=[0.95, 0.05])).astype(int)
  merged['Iron man'] = (merged['ironman'] - np.random.choice([0,manpower_regulator],
                                          len(merged), p=[0.95, 0.05])).astype(int)
  merged['QC'] = (merged['qc'] - np.random.choice([0,manpower_regulator],
                                          len(merged), p=[0.95, 0.05])).astype(int)

  merged['Man power'] = merged['Operator'] + merged['Helper'] + merged['Iron man'] + merged['QC']
  merged.drop(columns=['operator', 'helper', 'ironman', 'qc','manpower', 'qty', 'mins','produced_mins'], inplace=True)
  return merged

In [30]:
new_merged = generate_manpower_kpis(new_merged, manpower_regulator= 1)

In [31]:
new_merged = new_merged [['Date', 'Line', 'Hour', 'FTT', 'SMV', 'TPPM' , 'Defective' ,
                 'Defect' , 'Rectified' ,'Reject' , 'Operator' ,'Helper' , 'Iron man', 'QC',
                 'Man power']]

new_merged['Date'] = new_merged['Date'].dt.strftime('%Y-%m-%d')




<ipython-input-31-68f1fa585320>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_merged['Date'] = new_merged['Date'].dt.strftime('%Y-%m-%d')


In [32]:
new_merged

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Iron man,QC,Man power
0,2024-07-26,F1M7,1,186,8.28,3.04,2,4,0,3,23,5,5,4,37
1,2024-07-26,F1M7,2,187,8.28,3.04,9,12,7,2,23,5,5,4,37
2,2024-07-26,F1M7,3,192,8.28,3.04,5,6,4,3,23,5,5,4,37
3,2024-07-26,F1M7,4,190,8.28,3.04,2,5,0,2,23,5,5,4,37
4,2024-07-26,F1M7,5,188,8.28,3.04,4,6,4,0,23,5,5,4,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,120,8.28,2.04,2,4,0,3,14,6,5,4,29
150,2024-08-01,F1M8,8,126,8.28,2.04,5,6,4,1,14,6,5,4,29
151,2024-08-01,F1M8,9,127,8.28,2.04,5,9,4,2,14,6,5,4,29
152,2024-08-01,F1M8,10,128,8.28,2.04,5,5,3,2,14,6,5,3,28


In [33]:
new_merged.to_excel('synthetic_targets.xlsx', index=False)


df_from_new_excel = pd.read_excel('synthetic_targets.xlsx')

# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    df_from_new_excel.to_excel(writer, sheet_name='target_kpis', index=False)